# RAG

## Load model

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('flaubert/flaubert_base_uncased')

/Users/naowak/Thesis/code/SummaTalk/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name flaubert/flaubert_base_uncased. Creating a new one with MEAN pooling.


## Convert text to embeddings

In [2]:
import os
import pandas as pd

dir = '../texts/'
texts_files = os.listdir(dir) 
texts_embeddings = []
NB_LINES_PER_CHUNK = 50

for text in texts_files:
    # Read the text file
    with open(dir+text, 'r') as file:
        data = file.readlines()
    
    # Split the text into chunks of 5 lines
    chunks = [' '.join(data[i:i+NB_LINES_PER_CHUNK]) for i in range(0, len(data), 5)]
    embeddings = model.encode(chunks)

    # Store the embeddings
    for i, embed in enumerate(embeddings):
        entry = {
            'embedding': list(embed),
            'filename': text.split('.')[0],
            'text': chunks[i],
            'index': i,
        }
        texts_embeddings.append(entry)

df = pd.DataFrame(texts_embeddings)
df.to_csv('rag_db.csv', index=False)
df.head()

/var/folders/_8/z8rcf_ns4k9f2rw5zchj85mw0000gn/T/ipykernel_21580/1964331462.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,embedding,filename,text,index
0,"[0.7117877, 0.52825713, -0.50514823, -0.008762...",thinkerview-2,"Musique\n ...\n Messieurs bonjour, nous vou...",0
1,"[0.75147784, 0.33859283, -0.5631464, 0.1059062...",thinkerview-2,"Bonjour.\n Nicolas Méian, présentez-vous suc...",1
2,"[0.6059554, 0.471013, -0.52761763, 0.27381724,...",thinkerview-2,"Bonjour, Francis Perrin. Je dirige une sociét...",2
3,"[0.68403167, 0.23477107, -0.6883479, 0.1858753...",thinkerview-2,Accessoirement l'auteur d'un ouvrage sur la g...,3
4,"[0.71725535, 0.41893554, -0.3627441, 0.3586545...",thinkerview-2,Il y a plusieurs géopolitiques et plusieurs p...,4


## Find closest chunks

In [76]:
from sklearn.metrics.pairwise import cosine_similarity

# Load the vectorized database
df = pd.read_csv('rag_db.csv')
df.embedding = df.embedding.apply(lambda x: eval(x))

def find_closest_chunk(query, df, model, n=1):
    query_embedding = model.encode([query])[0]
    similarity = df.embedding.apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    ids = similarity.sort_values(ascending=False).head(10)
    return df.loc[ids.index].text.tolist()

In [77]:
find_closest_chunk("L'intelligence artificielle au service de l'humanité", df, model)

[" C'est assez flippant et très peu abordé.\n  Et vous pouvez voir une démonstration pour vous faire un avis\n  dans cette vidéo.\n  Sous-titrage Société Radio-Canada",
 " soit pour se balader sur Internet ou se connecter à vos mails,\n  vos documents, etc.\n  Le truc, c'est que beaucoup de gens ne réalisent pas\n  qu'il y a une vulnérabilité, une faille de sécurité\n  intrinsèque aux modèles de langage.\n  C'est assez flippant et très peu abordé.\n  Et vous pouvez voir une démonstration pour vous faire un avis\n  dans cette vidéo.\n  Sous-titrage Société Radio-Canada",
 " Et surtout, merci d'avoir regardé jusqu'au bout.\n  Bye bye.",
 " Oh putain\n  Moi je m'en aurais rentré dedans\n  Drama\n  Drama\n  Intelligence\n  Le retour\n  T'as l'entre deux\n  T'as 12 extrêmes\n  T'as Basti\n  Qui est un espèce d'emoji\n  Souriant sur patte\n  Et qui est gentil avec tout le monde\n  T'as\n  De l'autre côté extrême du cercle\n  T'as Anis\n  Dont le but dans sa carrière\n  C'est de speedrun\n  L